# Experiment with different binary I/O formats for pandas dataframes

Formats in the test:
- csv
- pickle
- feather
- parquet

Dataset for the test: 
- Transaction data with nearly 6 Mio. rows.
- dtypes: category(1), datetime64(1), float64(2), int64(4), object(1)


Results:
 - All binary formats are _way_ much faster at writing than csv
 - All binary formats keep dtype information (csv does not)
 - Parquet has much smaller filesize than csv, pickle and feather with standard compression don't gain whit this dataset
 - Pickle has the fastest reads, feather the fastest writes
 
Some addidional remarks:
 - Pickle and feather are not recommended for long-time storage
 - Parquet offers the option to read dfs only partially from disc (see very end of notebook)
 - Parquet also allows for distributed storage / computing (not tested here)
 - csv wins out on human readability and a large compatibility with other programs / languages
 
Blogposts with more information / tests:
- https://towardsdatascience.com/stop-persisting-pandas-data-frames-in-csvs-f369a6440af5
- https://towardsdatascience.com/the-best-format-to-save-pandas-data-414dca023e0d

In [1]:
import os
import random
import string
import pandas as pd

In [2]:
# Load and prepare df (original data is saved as csv, we define some special datatypes)

df = pd.read_csv("transactions.csv", parse_dates=["DateSK"], dtype={"Basiswarengruppe": "category"})

In [4]:
# Add an object column too with random strings of length 3-10

letters = string.ascii_letters
list_of_strings = [
    ''.join(random.choice(letters)
    for _ in range(random.randint(3, 10)))
    for _ in range(len(df))
]
df["String"] = list_of_strings

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5940559 entries, 0 to 5940558
Data columns (total 9 columns):
 #   Column              Dtype         
---  ------              -----         
 0   DateSK              datetime64[ns]
 1   Amount              float64       
 2   MemberAK            int64         
 3   SiteSK              int64         
 4   TransactionItemSK   int64         
 5   AnalysisCode1       int64         
 6   ist_umsatz_Loeb_mp  float64       
 7   Basiswarengruppe    category      
 8   String              object        
dtypes: category(1), datetime64[ns](1), float64(2), int64(4), object(1)
memory usage: 368.2+ MB


In [6]:
df.head(2)

,DateSK,Amount,MemberAK,SiteSK,TransactionItemSK,AnalysisCode1,ist_umsatz_Loeb_mp,Basiswarengruppe,String
0,2018-02-20,2.25,1235113,48,41203157,50104,1.0,Lebensmittel,HGaVffr
1,2018-10-19,6.95,1243518,49,41269022,16102,1.0,Damenunterbekleidung,PolNufwiKa


## CSV Format (baseline)

In [7]:
%timeit -n 3 df.to_csv("transactions_out.csv", index=False)

1min ± 2.42 s per loop (mean ± std. dev. of 7 runs, 3 loops each)


In [8]:
%timeit -n 3 df_csv = pd.read_csv("transactions.csv")

3.52 s ± 135 ms per loop (mean ± std. dev. of 7 runs, 3 loops each)


In [13]:
print(f"Filesize {os.path.getsize('transactions_out.csv') / 1024 :,.2f}")

Filesize 389,091.21


In [16]:
df_csv = pd.read_csv("transactions_out.csv")
df_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5940559 entries, 0 to 5940558
Data columns (total 9 columns):
 #   Column              Dtype  
---  ------              -----  
 0   DateSK              object 
 1   Amount              float64
 2   MemberAK            int64  
 3   SiteSK              int64  
 4   TransactionItemSK   int64  
 5   AnalysisCode1       int64  
 6   ist_umsatz_Loeb_mp  float64
 7   Basiswarengruppe    object 
 8   String              object 
dtypes: float64(2), int64(4), object(3)
memory usage: 407.9+ MB


## Pickle

[docs](https://docs.python.org/3/library/pickle.html)

In [17]:
import pickle

In [23]:
%%timeit -n 3
with open(f"transactions_out.pkl", "wb") as f:
    pickle.dump(df, f, pickle.HIGHEST_PROTOCOL)

3.06 s ± 72.2 ms per loop (mean ± std. dev. of 7 runs, 3 loops each)


In [19]:
%%timeit -n 3
with open(f"transactions_out.pkl", "rb") as f:
    df_pkl = pickle.load(f)

1.05 s ± 6.5 ms per loop (mean ± std. dev. of 7 runs, 3 loops each)


In [20]:
print(f"Filesize {os.path.getsize('transactions_out.pkl') / 1024 :,.2f}")

Filesize 385,800.03


In [22]:
with open(f"transactions_out.pkl", "rb") as f:
    df_pkl = pickle.load(f)
df_pkl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5940559 entries, 0 to 5940558
Data columns (total 9 columns):
 #   Column              Dtype         
---  ------              -----         
 0   DateSK              datetime64[ns]
 1   Amount              float64       
 2   MemberAK            int64         
 3   SiteSK              int64         
 4   TransactionItemSK   int64         
 5   AnalysisCode1       int64         
 6   ist_umsatz_Loeb_mp  float64       
 7   Basiswarengruppe    category      
 8   String              object        
dtypes: category(1), datetime64[ns](1), float64(2), int64(4), object(1)
memory usage: 368.2+ MB


## Feather

[docs](https://arrow.apache.org/docs/python/feather.html)

In [24]:
import pyarrow.feather as feather

%timeit -n 3 feather.write_feather(df, "transactions_out.feather")

1.06 s ± 15.5 ms per loop (mean ± std. dev. of 7 runs, 3 loops each)


In [25]:
%timeit -n 3 df_feather = feather.read_feather("transactions_out.feather")

3.05 s ± 28.6 ms per loop (mean ± std. dev. of 7 runs, 3 loops each)


In [26]:
print(f"Filesize {os.path.getsize('transactions_out.csv') / 1024 :,.2f}")

Filesize 389,091.21


In [27]:
df_feather = feather.read_feather("transactions_out.feather")
df_feather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5940559 entries, 0 to 5940558
Data columns (total 9 columns):
 #   Column              Dtype         
---  ------              -----         
 0   DateSK              datetime64[ns]
 1   Amount              float64       
 2   MemberAK            int64         
 3   SiteSK              int64         
 4   TransactionItemSK   int64         
 5   AnalysisCode1       int64         
 6   ist_umsatz_Loeb_mp  float64       
 7   Basiswarengruppe    category      
 8   String              object        
dtypes: category(1), datetime64[ns](1), float64(2), int64(4), object(1)
memory usage: 368.2+ MB


## Parquet

[docs](https://arrow.apache.org/docs/python/parquet.html)

In [30]:
import pyarrow as pa
import pyarrow.parquet as pq

In [36]:
%%timeit -n 3
table = pa.Table.from_pandas(df, preserve_index=False)  # index of no value here
pq.write_table(table, 'transactions_out.parquet')

2.64 s ± 67.1 ms per loop (mean ± std. dev. of 7 runs, 3 loops each)


In [37]:
%%timeit -n 3 
df_parquet = pq.read_pandas('transactions_out.parquet').to_pandas()

3.57 s ± 22.9 ms per loop (mean ± std. dev. of 7 runs, 3 loops each)


In [38]:
print(f"Filesize {os.path.getsize('transactions_out.parquet') / 1024 :,.2f}")

Filesize 137,552.14


In [40]:
df_parquet = pq.read_pandas('transactions_out.parquet').to_pandas()
df_parquet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5940559 entries, 0 to 5940558
Data columns (total 9 columns):
 #   Column              Dtype         
---  ------              -----         
 0   DateSK              datetime64[ns]
 1   Amount              float64       
 2   MemberAK            int64         
 3   SiteSK              int64         
 4   TransactionItemSK   int64         
 5   AnalysisCode1       int64         
 6   ist_umsatz_Loeb_mp  float64       
 7   Basiswarengruppe    category      
 8   String              object        
dtypes: category(1), datetime64[ns](1), float64(2), int64(4), object(1)
memory usage: 368.2+ MB


In [42]:
# Additonal test: Partial read only

df_part = pq.read_pandas("transactions_out.parquet", columns=["DateSK", "MemberAK", "String"]).to_pandas()

In [43]:
df_part.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5940559 entries, 0 to 5940558
Data columns (total 3 columns):
 #   Column    Dtype         
---  ------    -----         
 0   DateSK    datetime64[ns]
 1   MemberAK  int64         
 2   String    object        
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 136.0+ MB


In [44]:
# Work with direct pandas integration

df.to_parquet("transactions_out.parquet", 
              engine="pyarrow", 
              compression="snappy", 
              index=False, 
              partition_cols=None
)

In [46]:
print(f"Filesize {os.path.getsize('transactions_out.parquet') / 1024 :,.2f}")

Filesize 137,551.15


In [45]:
df_pandas = pd.read_parquet("transactions_out.parquet")

In [47]:
df_pandas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5940559 entries, 0 to 5940558
Data columns (total 9 columns):
 #   Column              Dtype         
---  ------              -----         
 0   DateSK              datetime64[ns]
 1   Amount              float64       
 2   MemberAK            int64         
 3   SiteSK              int64         
 4   TransactionItemSK   int64         
 5   AnalysisCode1       int64         
 6   ist_umsatz_Loeb_mp  float64       
 7   Basiswarengruppe    category      
 8   String              object        
dtypes: category(1), datetime64[ns](1), float64(2), int64(4), object(1)
memory usage: 368.2+ MB


---